In [1]:
# print best reward iteration and best reward for each training
%load_ext autoreload
%autoreload 2
import torch

available_gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]
print(available_gpus)
import gym
from gym.spaces import Discrete, MultiDiscrete, Box, MultiBinary, Dict
import numpy as np
import random
from ray.rllib.env.multi_agent_env import MultiAgentEnv
from MultiAgentSync_fullobs_samefield_randnose import MultiAgentSync_fullobs
from MultiAgentSync_fullobs_samefield_randnose import MultiAgentSing_fullobs
from scipy.io import loadmat
import os
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer

# from simple_rnn_v2_3_2_r import AnotherTorchRNNModel
from simple_rnn_v2_3_2 import AnotherTorchRNNModel

# from generic_fc_net import FullyConnectedNetwork
from ray.rllib.examples.models.rnn_model import RNNModel, TorchRNNModel
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.models import ModelCatalog
from ray.rllib.agents.ppo import PPOTorchPolicy
from Customcallback import CustomCallbacks

[<torch.cuda.device object at 0x7fe53b7f0340>, <torch.cuda.device object at 0x7fe53b7f12d0>]


/home/kumquat/miniconda3/envs/ray22/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


In [47]:
# single stage 
training_path = [f"/home/kumquat/Documents/Linfan/MARL/trainings/bigarena_single_randnose_rnn_n1_{i}" for i in range(1,11)]
for i in range(len(training_path)):
    path = training_path[i]

    if not os.path.exists(path):
        # The path does not exist, so skip this iteration
        continue
    analysis = tune.ExperimentAnalysis(path)
    data = analysis.trial_dataframes
    key = list(data.keys())[0]
    data = data[key]
    reward_array = np.array(data['episode_reward_mean'].values)
    indices = np.arange(9, len(reward_array), 10)
    selected_numbers = reward_array[indices]
    max_number = np.max(selected_numbers)
    max_iteration = np.where(reward_array == max_number)[0][0]
    print(f"training {i+1} best iteration {max_iteration+1} rewards {reward_array[max_iteration]}")

training 1 best iteration 4330 rewards 38.836999999999776
training 2 best iteration 4530 rewards 39.636999999999766
training 3 best iteration 6260 rewards 36.96899999999977
training 4 best iteration 5950 rewards 36.61399999999978
training 5 best iteration 5090 rewards 38.62799999999979
training 6 best iteration 3410 rewards 39.426999999999765
training 7 best iteration 4500 rewards 38.96399999999977
training 8 best iteration 3300 rewards 38.63999999999977
training 9 best iteration 5960 rewards 39.54599999999977
training 10 best iteration 4490 rewards 38.63799999999978


In [1]:
# coop stage 1
training_path = [f"/home/kumquat/Documents/Linfan/MARL/trainings/bigarena_sync_s1_randnose_rnn_n1_{i}.rewards_learn.mat" for i in range(1,11)]

In [31]:
for i in range(len(training_path)):
    path = training_path[i]

    if not os.path.exists(path):
        # The path does not exist, so skip this iteration
        continue

    # The path exists, so load the .mat file
    data = loadmat(path)
    file = loadmat(training_path[i])
    reward_array = np.array(file['rewards_learn'][0])
    indices = np.arange(9, len(reward_array), 10)
    selected_numbers = reward_array[indices]
    max_number = np.max(selected_numbers)
    max_iteration = np.where(reward_array == max_number)[0][0]
    formatted_index = str(max_iteration+1).zfill(6)
    path_res = os.path.join(path,"checkpoint_"+formatted_index)
    print(f"training {i+1} best iteration {max_iteration+1} rewards {reward_array[max_iteration]}")

training 1 best iteration 1130 rewards 15.990999999999962
training 3 best iteration 1020 rewards 19.178999999999938
training 4 best iteration 740 rewards 18.191999999999936
training 5 best iteration 650 rewards 20.53399999999992
training 6 best iteration 880 rewards 22.099999999999913
training 7 best iteration 810 rewards 23.78099999999989
training 8 best iteration 1010 rewards 22.277999999999903
training 9 best iteration 830 rewards 18.738999999999933
training 10 best iteration 1140 rewards 20.714999999999918


In [2]:
# coop stage 2
training_path = [f"/home/kumquat/Documents/Linfan/MARL/trainings/bigarena_sync_s2_randnose_rnn_n1_{i}.rewards_learn.mat" for i in range(1,11)]

In [3]:
for i in range(len(training_path)):
    path = training_path[i]

    if not os.path.exists(path):
        # The path does not exist, so skip this iteration
        continue

    # The path exists, so load the .mat file
    data = loadmat(path)
    file = loadmat(training_path[i])
    reward_array = np.array(file['rewards_learn'][0])
    indices = np.arange(9, len(reward_array), 10)
    selected_numbers = reward_array[indices]
    max_number = np.max(selected_numbers)
    max_iteration = np.where(reward_array == max_number)[0][0]
    formatted_index = str(max_iteration+1).zfill(6)
    path_res = os.path.join(path,"checkpoint_"+formatted_index)
    print(f"training {i+1} best iteration {max_iteration+1} rewards {reward_array[max_iteration]}")

training 1 best iteration 1690 rewards 8.60899999999998
training 2 best iteration 2290 rewards 12.369999999999981
training 3 best iteration 1710 rewards 11.640999999999973
training 4 best iteration 5470 rewards 13.08099999999997
training 5 best iteration 750 rewards 12.356999999999985
training 6 best iteration 1880 rewards 13.343999999999976
training 7 best iteration 2200 rewards 15.388999999999953
training 8 best iteration 2390 rewards 15.70299999999995
training 9 best iteration 2910 rewards 14.14199999999998
training 10 best iteration 1830 rewards 14.103999999999958


In [2]:
## generate video roll outs 

idx = [2, 3, 4, 5, 6, 7, 8, 9, 10]
training_path = [
    f"/home/kumquat/Documents/Linfan/MARL/trainings/bigarena_sync_s2_randnose_rnn_n1_{i}"
    for i in idx
]
checkpoint_dirs = list()
for i in range(len(training_path)):
    path = training_path[i]

    if not os.path.exists(path):
        # The path does not exist, so skip this iteration
        continue
    # The path exists, so load the .mat file
    file = loadmat(f"{path}.rewards_learn.mat")
    reward_array = np.array(file["rewards_learn"][0])
    indices = np.arange(9, len(reward_array), 10)
    selected_numbers = reward_array[indices]
    max_number = np.max(selected_numbers)
    max_iteration = np.where(reward_array == max_number)[0][0]
    formatted_index = str(max_iteration + 1).zfill(6)
    checkpoint_dirs.append(os.path.join(path, "checkpoint_" + formatted_index))
    print(
        f"training {idx[i]} best iteration {max_iteration+1} rewards {reward_array[max_iteration]}"
    )

training 2 best iteration 2290 rewards 12.369999999999981
training 3 best iteration 1710 rewards 11.640999999999973
training 4 best iteration 5470 rewards 13.08099999999997
training 5 best iteration 750 rewards 12.356999999999985
training 6 best iteration 1880 rewards 13.343999999999976
training 7 best iteration 2200 rewards 15.388999999999953
training 8 best iteration 2390 rewards 15.70299999999995
training 9 best iteration 2910 rewards 14.14199999999998
training 10 best iteration 1830 rewards 14.103999999999958


In [3]:
from current_config import current_coop_config, current_sing_config

coop_config, env_config = current_coop_config()
sing_config, env_config = current_sing_config()


agent1 is now mapped to policy1
agent2 is now mapped to policy2

agent1 is now mapped to policy1
agent2 is now mapped to policy2


In [21]:
agent = PPOTrainer(config=coop_config)
agent.restore(checkpoint_dirs[8])

2023-11-20 15:35:37,513	WARNING catalog.py:637 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
2023-11-20 15:35:37,527	WARNING catalog.py:637 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
2023-11-20 15:35:37,540	WARNING util.py:66 -- Install gputil for GPU system monitoring.
2023-11-20 15:35:37,554	INFO trainable.py:790 -- Restored on 10.47.57.245 from checkpoint: /home/kumquat/Documents/Linfan/MARL/trainings/bigarena_sync_s2_randnose_rnn_n1_10/checkpoint_001830
2023-11-20 15:35:37,554	INFO trainable.py:799 -- Current state after restoring: {'_iteration': 1830, '_timesteps_total': None, '_time_total': 57882.686299562454, '_episodes_total': 36600}


In [22]:
import time
from ipywidgets import Output
from IPython import display

out = Output()
display.display(out)

with out:
    env = MultiAgentSync_fullobs(config=env_config)
    env.timestep_limit = 200
    obs = env.reset()
    state1 = [np.zeros(256, np.float32) for _ in range(2)]
    state2 = [np.zeros(256, np.float32) for _ in range(2)]
    while True:
        a1, state1, _ = agent.compute_single_action(
            observation=obs["agent1"], state=state1, policy_id="policy1"
        )
        #
        a2, state2, _ = agent.compute_single_action(
            observation=obs["agent2"], state=state2, policy_id="policy2"
        )
        obs, rewards, dones, _ = env.step({"agent1": a1, "agent2": a2})
        # print(env.agent1_pos)
        print(a2)
        out.clear_output(wait=True)
        time.sleep(0.2)
        env.render()
        if dones["agent1"]:
            break

Output()

In [13]:
# cross pair roll out videos
agent1 = PPOTrainer(config=config)
agent1.restore(checkpoint_dirs[0]) # pol2
agent2 = PPOTrainer(config=config)
agent2.restore(checkpoint_dirs[4]) # pol2

2023-11-16 08:56:40,251	WARNING catalog.py:637 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
2023-11-16 08:56:40,270	WARNING catalog.py:637 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
2023-11-16 08:56:40,287	WARNING util.py:66 -- Install gputil for GPU system monitoring.
2023-11-16 08:56:40,301	INFO trainable.py:790 -- Restored on 10.47.57.245 from checkpoint: /home/kumquat/Documents/Linfan/MARL/trainings/bigarena_sync_s2_randnose_rnn_n1_2/checkpoint_002290
2023-11-16 08:56:40,302	INFO trainable.py:799 -- Current state after restoring: {'_iteration': 2290, '_timesteps_total': None, '_time_total': 72203.9035396576, '_episodes_total': 45800}
2023-11-16 08:56:40,308	WARNING catalog.py:637 -- Custom ModelV2 should accept all custom options as **kwargs, instead of expecting them in config['custom_model_config']!
2023-11-16 08:56:40,323

In [14]:
import time
from ipywidgets import Output
from IPython import display

out = Output()
display.display(out)

with out:
    env = MultiAgentSync_fullobs(config=env_config)
    env.timestep_limit = 200
    obs = env.reset()
    state1 = [np.zeros(256, np.float32) for _ in range(2)]
    state2 = [np.zeros(256, np.float32) for _ in range(2)]
    while True:
        a1, state1, _ = agent1.compute_single_action(
            observation=obs["agent1"], state=state1, policy_id="policy2"
        )
        #
        a2, state2, _ = agent2.compute_single_action(
            observation=obs["agent2"], state=state2, policy_id="policy2"
        )
        obs, rewards, dones, _ = env.step({"agent1": a1, "agent2": a2})
        # print(env.agent1_pos)
        print(a2)
        out.clear_output(wait=True)
        time.sleep(0.2)
        env.render()
        if dones["agent1"]:
            break

Output()

In [24]:
agent.get_weights()

{'policy1': {'rnn.weight_ih_l0': array([[ 0.14933926,  0.7762351 ,  0.7750816 , ...,  0.94712114,
           0.89617676, -0.32242197],
         [-0.21022284, -0.66234934, -0.39191335, ..., -1.0066723 ,
          -0.45398572, -0.8082298 ],
         [-0.02859231, -0.27843088, -0.55727303, ..., -0.6746898 ,
          -1.148003  , -0.54113126],
         ...,
         [-0.6640275 ,  0.20968397, -0.00942521, ..., -0.18914728,
          -0.6384093 ,  0.09915647],
         [ 0.45563215,  0.8499066 ,  0.36223164, ...,  0.70111644,
           0.48669875, -1.9842869 ],
         [ 0.58573276, -0.0622965 , -0.3544612 , ...,  0.17647472,
           1.1323193 , -0.0300257 ]], dtype=float32),
  'rnn.weight_hh_l0': array([[ 0.01105353,  0.00136312, -0.00053385, ..., -0.00111275,
          -0.00318657, -0.00051168],
         [ 0.00501137,  0.00657068, -0.00092775, ...,  0.00334354,
           0.0045673 , -0.0019837 ],
         [ 0.00503891,  0.00520063,  0.00394853, ...,  0.00256934,
          -0.000590